**Utilities**

In [1]:
import re
from collections import deque
from copy import deepcopy

# I made this regex more brittle by encoding the instruction format directly into it. This ensures that any change to the source format will break the code, causing it to fail whatever tests would be written for it. Good interface design for something as flimsy as regex (or shell, etc.).
instr_regex = re.compile(r'^move\s*(?P<mult>\d+)\s*from\s*(?P<source>\d+)\s*to\s*(?P<target>\d+)$')

# Rather than make this a class or function, I'll just run it as a script. Too many public variables used in other contexts after it's completed.
with open('../inputs/day5-input') as f:
    """ Find the table in the file """
    rows = 0
    line = f.readline()
    while not line.lstrip()[0].isdigit():
        rows += 1
        line = f.readline()

    """ Read the table into a data structure """
    col_offsets = [_.start() for _ in re.finditer(re.compile(r'\d'), line)]
    stop = f.tell()  # We've read far enough to capture the table
    f.seek(0)
    table_raw = f.readlines(stop)[:rows]
    table = [[row[_] for _ in col_offsets] for row in table_raw]

    queue_list = [deque() for _ in col_offsets]
    for r in table:
        for i, c in enumerate(r):
            if c != ' ':
                queue_list[i].appendleft(c)

**Part 1** Follow the instructions and see what ends up on top

In [2]:
pt1_list = deepcopy(queue_list)

""" Follow the instructions """
with open('../inputs/day5-input') as f:
    f.seek(stop) # Return to the line after the table and its legend
    while True:
        line = f.readline()
        if len(line) != 0:  # Check for EOF
            instruction = re.match(instr_regex, line.strip())
            if instruction:  # Check for a non-instruction line
                details = instruction.groupdict()
                for _ in range(int(details['mult'])):
                    pt1_list[int(details['target']) - 1].append(pt1_list[int(details['source']) - 1].pop())
        else:
            break

print('Crates on top: ' + ''.join([_.pop() for _ in pt1_list]))

Crates on top: SHMSDGZVC


**Part 2**: Move several crates at once, preserving order

In [6]:
pt2_list = deepcopy(queue_list)
crane_queue = deque()  # Intermediate Q for holding a stack of crates to be moved all together

""" Follow the instructions """
with open('../inputs/day5-input') as f:
    f.seek(stop) # Return to the line after the table and its legend
    while True:
        line = f.readline()
        if len(line) != 0:  # Check for EOF
            instruction = re.match(instr_regex, line.strip())
            if instruction:  # Check for a non-instruction line
                details = instruction.groupdict()
                it = range(int(details['mult']))
                for _ in it:
                    crane_queue.append(pt2_list[int(details['source']) - 1].pop())
                for _ in it:
                    pt2_list[int(details['target']) - 1].append(crane_queue.pop())
        else:
            break

print('Crates on top: ' + ''.join([_.pop() for _ in pt2_list]))

Crates on top: VRZGHDFBQ
